In [1]:
!python -c "import monai" || pip install -q "monai-weekly[nibabel]"
!python -c "import matplotlib" || pip install -q matplotlib
%matplotlib inline

In [7]:
import os
import json
import shutil
import tempfile
import time

import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib

from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai import transforms
from monai.transforms import (
    AsDiscrete,
    Activations,
)

from monai.config import print_config
from monai.metrics import DiceMetric
from monai.utils.enums import MetricReduction
from monai.networks.nets import SwinUNETR
from monai import data
from monai.data import decollate_batch
from functools import partial

from monai.data import (
    ThreadDataLoader,
    CacheDataset,
    load_decathlon_datalist,
    decollate_batch,
    set_track_meta,
)

from monai.transforms import (
    AsDiscrete,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandFlipd,
    RandCropByPosNegLabeld,
    RandShiftIntensityd,
    ScaleIntensityRanged,
    Spacingd,
    RandRotate90d,
    EnsureTyped,
)

import torch


print_config()

MONAI version: 1.1.0
Numpy version: 1.21.6
Pytorch version: 1.13.1+cu117
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: a2ec3752f54bfc3b40e7952234fbeb5452ed63e3
MONAI __file__: /home/dhruvjignesh.shah001/anaconda3/envs/O/lib/python3.9/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.10
Nibabel version: 5.0.1
scikit-image version: 0.19.2
Pillow version: 9.0.1
Tensorboard version: 2.8.0
gdown version: 4.6.3
TorchVision version: 0.14.1+cu117
tqdm version: 4.64.1
lmdb version: 1.4.0
psutil version: 5.9.1
pandas version: 1.2.3
einops version: 0.6.0
transformers version: 4.21.3
mlflow version: 2.1.1
pynrrd version: 1.0.0

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [3]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

/tmp/tmpr5ib3pwo


In [27]:
data_path = "research-contributions/Nathan Arnett Calcification/"

In [28]:
 def load(which, datapath=data_path, with_header=False):

    DATASETS = sorted([v for v in os.listdir(datapath) if os.path.isdir(datapath + v)])

    D = DATASETS[which]
    print(D)

    files = os.listdir(datapath + D)
    label_file = [os.path.join(datapath, D, s) for s in files if 'seg' in s]
    image_file = [os.path.join(datapath, D, i) for i in files if not 'seg' in i]
    label_file = [s for s in label_file if os.path.isfile(s)][0]
    image_file = [i for i in image_file if os.path.isfile(i)][0]

    label_with_header = nrrd.read(label_file)
    image_with_header = nrrd.read(image_file)

    label = label_with_header[0]
    image = image_with_header[0]

    if with_header:
      return image_with_header, label_with_header

    return image, label

In [29]:
def parse_folder(folder):
    
    image = []
    segmentation = []
    prnt_folder  = os.listdir(folder)
    for i in prnt_folder:
        path = folder + '/' + i
        sub_files = os.listdir(path)
        print(sub_files)
        print(path)

        size1 = os.path.getsize(path+'/'+sub_files[0])
        size2 = os.path.getsize(path+'/'+sub_files[1])
        
        if size1 < size2:
            segmentation.append(sub_files[0])
            image.append(sub_files[1])
        else:
            segmentation.append(sub_files[1])
            image.append(sub_files[0])
    
    return image, segmentation

In [31]:
image, segmentation = parse_folder(data_path)

['Segmentation.seg.nrrd', '12 Unnamed Series.nrrd']
research-contributions/Nathan Arnett Calcification//DICOM 40
['Segmentation.seg.nrrd', '4 Unnamed Series.nrrd']
research-contributions/Nathan Arnett Calcification//DICOM 96
['Segmentation.seg.nrrd', '4 Unnamed Series.nrrd']
research-contributions/Nathan Arnett Calcification//DICOM 72
['5 Unnamed Series.nrrd', 'Segmentation.seg.nrrd']
research-contributions/Nathan Arnett Calcification//DICOM 24
['Segmentation.seg.nrrd', '4 Unnamed Series.nrrd']
research-contributions/Nathan Arnett Calcification//DICOM 16
['Segmentation.seg.nrrd', '4 Unnamed Series.nrrd']
research-contributions/Nathan Arnett Calcification//DICOM 61
['Segmentation.seg.nrrd', '7 Unnamed Series.nrrd']
research-contributions/Nathan Arnett Calcification//DICOM 88
['Segmentation.seg.nrrd', '6 Unnamed Series.nrrd']
research-contributions/Nathan Arnett Calcification//DICOM 82
['Segmentation.seg.nrrd', '12 Unnamed Series.nrrd']
research-contributions/Nathan Arnett Calcification/

In [9]:
import nrrd

In [17]:
import numpy as np

def pad(image, label):
    maxX = 512
    maxY = 512
    maxZ = 384
    slicecount = 0
    
    padded_images = np.zeros((slicecount, maxY, maxX, maxZ), dtype=image[0].dtype)
    padded_labels = np.zeros((slicecount, maxY, maxX, maxZ), dtype=label[0].dtype)

    img = image
    lbl = label
        
        # Calculate the amount of padding needed
    pad_x = maxX - img.shape[1]
    pad_y = maxY - img.shape[0]
    pad_z = maxZ - img.shape[2]
    print("-----------------------")
    print(pad_x)
    print(pad_z)
        
        # Pad the image and label
    padded_img = np.pad(img, ((0, pad_y), (0, pad_x), (0, pad_z)), 'constant')
    padded_lbl = np.pad(lbl, ((0, pad_y), (0, pad_x), (0, pad_z)), 'constant')
    print("-----------------------")
    print(padded_img)
    padded_images = padded_img
    padded_labels = padded_lbl
        
    return padded_images, padded_labels


In [32]:
image, label = load(0)
padded_image, padded_label = pad(image, label)
nrrd.write('Image_'+str(0)+'.nrrd', padded_image)
nrrd.write('Label_'+str(0)+'.nrrd', padded_label)

DICOM 12
-----------------------
0
162
-----------------------
[[[-1010 -1008  -998 ...     0     0     0]
  [-1010 -1012  -996 ...     0     0     0]
  [-1008 -1014  -995 ...     0     0     0]
  ...
  [ -789  -787  -802 ...     0     0     0]
  [ -613  -589  -582 ...     0     0     0]
  [ -361  -344  -343 ...     0     0     0]]

 [[ -999 -1015 -1008 ...     0     0     0]
  [-1009 -1016 -1006 ...     0     0     0]
  [-1011 -1016 -1005 ...     0     0     0]
  ...
  [ -806  -807  -816 ...     0     0     0]
  [ -681  -663  -661 ...     0     0     0]
  [ -458  -451  -445 ...     0     0     0]]

 [[ -991 -1013 -1013 ...     0     0     0]
  [-1002 -1014 -1010 ...     0     0     0]
  [-1011 -1015 -1005 ...     0     0     0]
  ...
  [ -797  -815  -809 ...     0     0     0]
  [ -736  -734  -731 ...     0     0     0]
  [ -570  -568  -562 ...     0     0     0]]

 ...

 [[-1001 -1004 -1000 ...     0     0     0]
  [-1001 -1004  -994 ...     0     0     0]
  [-1000 -1006  -998 ...   

In [19]:
data, header = nrrd.read('Image_0.nrrd')
slice_index = 0


# Display the slice using matplotlib
#plt.imshow(image_slice, cmap='gray')
#plt.show()

In [20]:
print(data.shape)

(512, 512, 384)


In [21]:
print(data)

[[[-1010 -1008  -998 ...     0     0     0]
  [-1010 -1012  -996 ...     0     0     0]
  [-1008 -1014  -995 ...     0     0     0]
  ...
  [ -789  -787  -802 ...     0     0     0]
  [ -613  -589  -582 ...     0     0     0]
  [ -361  -344  -343 ...     0     0     0]]

 [[ -999 -1015 -1008 ...     0     0     0]
  [-1009 -1016 -1006 ...     0     0     0]
  [-1011 -1016 -1005 ...     0     0     0]
  ...
  [ -806  -807  -816 ...     0     0     0]
  [ -681  -663  -661 ...     0     0     0]
  [ -458  -451  -445 ...     0     0     0]]

 [[ -991 -1013 -1013 ...     0     0     0]
  [-1002 -1014 -1010 ...     0     0     0]
  [-1011 -1015 -1005 ...     0     0     0]
  ...
  [ -797  -815  -809 ...     0     0     0]
  [ -736  -734  -731 ...     0     0     0]
  [ -570  -568  -562 ...     0     0     0]]

 ...

 [[-1001 -1004 -1000 ...     0     0     0]
  [-1001 -1004  -994 ...     0     0     0]
  [-1000 -1006  -998 ...     0     0     0]
  ...
  [ -906  -898  -920 ...     0     0    

In [33]:
for i in range(35):
    image, label = load(i)
    padded_image, padded_label = pad(image, label)
    nrrd.write('paddedImages/Image_' + str(i) + '.nrrd', padded_image)
    nrrd.write('paddedImages/Label_' + str(i) + '.nrrd', padded_label)

DICOM 12
-----------------------
0
162
-----------------------
[[[-1010 -1008  -998 ...     0     0     0]
  [-1010 -1012  -996 ...     0     0     0]
  [-1008 -1014  -995 ...     0     0     0]
  ...
  [ -789  -787  -802 ...     0     0     0]
  [ -613  -589  -582 ...     0     0     0]
  [ -361  -344  -343 ...     0     0     0]]

 [[ -999 -1015 -1008 ...     0     0     0]
  [-1009 -1016 -1006 ...     0     0     0]
  [-1011 -1016 -1005 ...     0     0     0]
  ...
  [ -806  -807  -816 ...     0     0     0]
  [ -681  -663  -661 ...     0     0     0]
  [ -458  -451  -445 ...     0     0     0]]

 [[ -991 -1013 -1013 ...     0     0     0]
  [-1002 -1014 -1010 ...     0     0     0]
  [-1011 -1015 -1005 ...     0     0     0]
  ...
  [ -797  -815  -809 ...     0     0     0]
  [ -736  -734  -731 ...     0     0     0]
  [ -570  -568  -562 ...     0     0     0]]

 ...

 [[-1001 -1004 -1000 ...     0     0     0]
  [-1001 -1004  -994 ...     0     0     0]
  [-1000 -1006  -998 ...   

IndexError: list index out of range